In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [3]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

In [6]:
sentenData = spark.createDataFrame([
    (0.0, 'Hi i header about Spark'),
    (0.0, 'I wish java could use case classes'),
    (1.0, 'Logistic regression models are neat')
],['label','sentence'])

In [7]:
sentenData.show()

+-----+--------------------+
|label|            sentence|
+-----+--------------------+
|  0.0|Hi i header about...|
|  0.0|I wish java could...|
|  1.0|Logistic regressi...|
+-----+--------------------+



In [8]:
tokenizer = Tokenizer(inputCol='sentence', outputCol='words')

In [9]:
words_data = tokenizer.transform(sentenData)

In [10]:
words_data.show(truncate=False)

+-----+-----------------------------------+------------------------------------------+
|label|sentence                           |words                                     |
+-----+-----------------------------------+------------------------------------------+
|0.0  |Hi i header about Spark            |[hi, i, header, about, spark]             |
|0.0  |I wish java could use case classes |[i, wish, java, could, use, case, classes]|
|1.0  |Logistic regression models are neat|[logistic, regression, models, are, neat] |
+-----+-----------------------------------+------------------------------------------+



In [11]:
hashing_tf = HashingTF(inputCol='words', outputCol='rawFeatures')

In [12]:
featurized_data = hashing_tf.transform(words_data)

In [13]:
idf = IDF(inputCol='rawFeatures', outputCol='features')

In [14]:
idf_model = idf.fit(featurized_data)

In [15]:
rescaled_data = idf_model.transform(featurized_data)

In [18]:
rescaled_data.select('label', 'features').show(truncate=False)

+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|features                                                                                                                                                                                        |
+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0.0  |(262144,[24417,49304,91137,161248,234657],[0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453])                                                    |
|0.0  |(262144,[20719,24417,55551,116873,147765,162369,192310],[0.6931471805599453,0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.693147180559945

In [19]:
from pyspark.ml.feature import CountVectorizer

In [20]:
df = spark.createDataFrame([
    (0, 'a b c'.split(' ')),
    (1, 'a b b c a'.split(' '))
],['id', 'words'])

In [21]:
df.show()

+---+---------------+
| id|          words|
+---+---------------+
|  0|      [a, b, c]|
|  1|[a, b, b, c, a]|
+---+---------------+



In [22]:
cv = CountVectorizer(inputCol='words', outputCol='features',
                    vocabSize=3, minDF=2.0)

In [23]:
model = cv.fit(df)

In [24]:
result = model.transform(df)

In [26]:
result.show(truncate=False)

+---+---------------+-------------------------+
|id |words          |features                 |
+---+---------------+-------------------------+
|0  |[a, b, c]      |(3,[0,1,2],[1.0,1.0,1.0])|
|1  |[a, b, b, c, a]|(3,[0,1,2],[2.0,2.0,1.0])|
+---+---------------+-------------------------+

